## Processing of custom area using LiCSAR (manual set)
*(based on COMET InSAR Course 2018, by K. Spaans, N. Greenall and M. Lazecky, Leeds University)*  
  
  
Supposing you have properly installed licsar_proc. If not, check this notebook: [licsar_proc_install](licsar_proc_install.ipynb)
### 1. Definition of area of interest and data sources
The polygon file is a text file which consists of a list of longitude and latitude coordinates which define the polygon of the processing frame. We currently use the extension .xy for polygon files. Currently the polygon defined is a rectangle and 5 coordinates must be defined, with the first and last coordinates being the same. See the example of *AOI.xy* file (covering Amatrice area):
```
12.7 42.1
14.3 42.1
14.3 43.4
12.7 43.4
12.7 42.1
```
A zip file list should be introduced as a text file which lists the absolute path to the Sentinel-1 SLC products contained in individual zip files. By convention we call the zip file *zipfile.list*. You may use more zip files of the same date, to be merged together:
```
/nfs/a1/insar_course/shared_data/SLC/S1A_IW_SLC__1SDV_20160821T051116_20160821T051143_012694_013F33_53E5.zip
/nfs/a1/insar_course/shared_data/SLC/S1A_IW_SLC__1SDV_20160821T051141_20160821T051209_012694_013F33_D1A8.zip
/nfs/a1/insar_course/shared_data/SLC/S1B_IW_SLC__1SDV_20160827T051035_20160827T051102_001798_002B0D_2E7D.zip
```
### 2. Data preprocessing
First of all we will **extract frame images**. This step is to prepare (merge, crop,..) the relevant .slc gamma-ready files, update their orbit information using precise ephemerides and create .mli (*multilook intensity*) previews. To do this we use the *LiCSAR_01_mk_images.py* tool:
```
LiCSAR_01_mk_images.py -d WORKDIR -p POLYGON.xy -z ZIPFILE.list -y1 -m MASTER -a AMLF -r RMLF
```
where WORKDIR is the working directory for the processing, POLYGON.xy is your polygon file, ZIPFILE.list is your zip file list, MASTER is the acquisition date of the primary image (master) in the format YYYYMMDD, AMLF is the integer azimuth multilook factor and RMLF is the range multilook factor. A directory called SLC will be created containing raw .slc files and .mli previews.  
In this case this will be:

In [ ]:
LiCSAR_01_mk_images.py -d $LiCSAR_temp -p AOI.xy -z zipfile.list -y1 -m 20160821 -a 3 -r 7

This command will process the frame crop defined by the polygon in AOI.xy with the files listed in zipfile.list. The master date is set to 21/08/2016 and the azimuth multilook factor is 3 and the range multilook factor is 7.  
*This step will uncompress the master files. In this case, the bursts are spread out over two files. It will then extract the relevant bursts and merge them into a new SLC, which will be used for further processing.*
### 3. Generation of DEM, geocoding tables and coregistration
The next step is to create a cropped DEM for our frame. Topography information is used for purposes of a more precise geocoding, coregistration and later on for removal of topographic effect from interferograms. The SRTM DEM can be automatically prepared using a shell script **LiCSAR_01_mk_crop_extDEM**  
The next step is to geocode the master image and coregister the slave images to the master. This will create a "geo" directory containing the master radar geocoded geometry height and multilooked intensity files. Another directory called RSLC will be created which contains the resampled coregistered slc files (.rslc). There will be some preview .bmp files in these directories. The multilook settings are as defined in the first step.
For this step, the LiCSAR_02_coreg.py tool is used:
```
LiCSAR_02_coreg.py -d WORKDIR -f FRAME -p POLYGON.xy -z ZIPFILE.list -y1 -m MASTER
```
where FRAME is the frame to be processed. If providing a polygon file, this will be the polygons file name without the extension.  
Extending our example:

In [ ]:
LiCSAR_02_coreg.py -d $LiCSAR_temp -f AOI -p AOI.xy -z zipfile.list -y 1 -m 20160821

You may display geo/EQA.dem.ras to see a geocoded preview of topography mixed with the multilooked intensity map. The image represents the height of the DEM, wrapped to 500 meters (in other words, every “fringe” or color cycle represents 500 meters of topography here). It is always good to check the DEM actually makes sense, as it plays a vital role during the coregistration and further processing steps.  
You can also check the coregistration result – a resampled secondary image (RSLC/20160827). All RSLC images should perfectly fit with the primary SLC image. You may not see how perfectly they fit – the fitting needs accuracy is not in a pixel level (that would be ok for non-interferometric application, not even in a tenth of pixel (that would be enough for any typical stripmap SAR imagery)
### 4. Generation of interferograms
The final steps are to create and unwrap interferograms between nearby dates. This will result in a folder called IFG, which contains interferograms (i.e. an interferogram between a pair of dates).
The tool used to do this is the LiCSAR_03_mk_ifgs.py tool:
```
LiCSAR_03_mk_ifgs.py -d WORKDIR -f FRAMENAME -p POLYGON.xy -z ZIPFILE.list -y1
```
In our example, the following command would be used:

In [ ]:
LiCSAR_03_mk_ifgs.py -d `pwd` -f AOI -p AOI.xy -z zipfile.list -y 1

To spatially filter and unwrap the generated interferograms we use the tool LiCSAR_04_unwrap.py:
```
LiCSAR_04_unwrap.py -d WORKDIR -f FRAMENAME -p POLYGON.xy -z ZIPFILE.list -y1
```
This will produce filtered interferograms under the file type .filt.diff and unwrapped interferograms under the file type .unw with associated .ras previews.  
Continuing our example:

In [ ]:
LiCSAR_04_unwrap.py -d `pwd` -f AOI -p AOI.xy -z zipfile.list -y 1

To regenerate the preview image with custom parameters, you may use a gamma function *rasrmg*, the original preview has been actually generated by:

In [ ]:
rasrmg IFG/20160821_20160827/20160821_20160827.unw  SLC/20160821/20160821.slc.mli 6980 1 1 0 1 1 0.3333

### 5. Geocoding the result
The last step transforms our results from the radar geometry we have seen so far to a regular latitude-longitude grid based on EQA.dem we saw during step 4. To generate a GeoTIFF file, you may use a script create_geoctiff_unw.sh :


In [ ]:
create_geoctiff_unw.sh $LiCSAR_temp 20160821 20160821_20160827